In [ ]:
#####################################################################################
# Google Places API
# - Google Places에서 평점, 리뷰수를 가져와서 csv로 저장한다.
#####################################################################################
import pandas as pd
import requests

# Google map API KEY
MY_API_KEY = ""

# 요청 URL
REQUEST_SEARCH_URL = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?key="+MY_API_KEY+"&input={keyword}&inputtype=textquery&fields=name,place_id"
REQUEST_DETAIL_URL = "https://maps.googleapis.com/maps/api/place/details/json?key="+MY_API_KEY+"&place_id={place_id}&language=ko"

# 평점있는 데이터와 없는 데이터 각 배열에 저장
rating_data = []
no_result = []

# csv 파일 경로
FILE_PATH = "관광지_분류위도경도_0120.csv"

# API 호출 후 JSON return한다.
def get_data(url, params):
    reqUrl = url.format(**params)
    print("REQUEST URL :", reqUrl)
    result = requests.get(reqUrl).json()
    
    return result

# DETAIL을 요청하여 리뷰수,별점 가져온다.
# place_id : google api에서의 관광지의 place ID
def get_rating(place_id):
    json_data = get_data(REQUEST_DETAIL_URL, {"place_id" : place_id})
    data = json_data["result"]
    
    if len(data) > 0 and "rating" in data:
        rating = data["rating"] # 평점
        review_count = data["user_ratings_total"] # 리뷰개수
        
        return (rating, review_count)
        
    else:
        return (None, None)

# 실행
def main():
    csv_data = pd.read_csv(FILE_PATH,
                           names=["분류", "장소명", "카카오맵_장소명", "검색_장소명", "위도", "경도", "주소"], 
                           header=0, encoding="euc-kr")

    for i in range(len(csv_data)):
        location = csv_data.iloc[i]
        location_name = location["검색_장소명"]
        category_name = location["분류"]

        # '제주' 지역이름을 붙여서 검색하여 조금이라도 정확하게 장소검색이 되도록..ㅠㅠ
        if location_name.find("제주") < 0:
            location_name = "제주" + location_name

        # DETAIL을 요청하기 위한 place_id 얻어오기
        json_data = get_data(REQUEST_SEARCH_URL, {"keyword" : location_name})
        data = json_data["candidates"]

        if len(data) > 0:
            place_id = data[0]["place_id"]
            rating, review_count = get_rating(place_id) # 별점,리뷰수
            
            if rating == None:
                no_result.append((category_name, location_name))
                print("검색결과에서 평점없음->", category_name, location_name)    
            else:
                rating_data.append((category_name, location_name, rating, review_count))
                print("->", category_name,"\t" , location_name,"\t" , rating,"\t" , review_count)
            
        else:
            no_result.append((category_name, location_name))
            print("결과없음->", category_name, location_name)

    # 평점 데이터, 결과없는 데이터 dataframe 생성
    rating_data_df = pd.DataFrame(rating_data, columns=["분류", "평점검색_장소명", "평점", "리뷰개수"])
    no_result_df = pd.DataFrame(no_result, columns=["분류", "평점검색_장소명"])
    
    print(rating_data_df)
    print()
    print(no_result)
                                                        
    # csv 파일로 저장
    if len(rating_data_df) > 0:
        rating_data_df.to_csv("rating_result.csv", 
                              mode="w", 
                              header=True, 
                              index=False,
                              encoding="euc-kr")
        
    if len(no_result_df) > 0:
        no_result_df.to_csv("rating_no_result.csv", 
                            mode="w", 
                            header=True, 
                            index=False,
                            encoding="euc-kr")

    print("끝")

            
# program start
main()
